# TensorFlow-Gammon

Wie im letzten Notebook gezeigt, spielt der TD-Gammon Spieler schon sehr vernünftig, wenn man das neuronale Netz mithilfe von Eligibility Traces einstellt. Was passiert aber, wenn wir Tensorflow die Gewichte selber einstellen lassen?

Zunächst verallgemeinere ich den __TD-GammonPlayer__ zu einem __ModelPlayer__ und erstelle eine __NeuralNetModel__ Super-Klasse um leichter Varianten erstellen zu können.

### ModelPlayer

Die einzige Änderung ist, dass der ModelPlayer nun den jewiligen Namen des Models trägt

In [48]:
from Player import ValuePlayer

class ModelPlayer(ValuePlayer):
    
    def __init__(self, player, model):
        ValuePlayer.__init__(self, player, self.get_value)
        self.model = model
        
    def get_value(self, game, player):
        features = game.extractFeatures(player)
        v = self.model.get_output(features)
        v = 1 - v if self.player == game.players[0] else v
        return v
    
    def get_name(self):
        return "ModelPlayer [" + self.model.get_name() +"]"

### NeuralNetModel

Die __NeuralNetModel__ Klasse enthält die nützlichen Funktion aus dem TDGammonModel (siehe Notebook 004) und lässt sich durch das einstellen der Variablen verändern bzw. anderes leicht erweitern.

Folgende Methoden müssen von Subklassen implementiert werden:_

```python
def creat_training_op(self)
```
<ul type="circle">
    <li>Diese Methode muss eine TensorFlow Trainings Methode bzw. eine Gruppe davon zurückgeben</li>
    <li>Hier wird entschieden wie die gewichte angepasst werden</li>
    <li>Beispiel:</li>
        <ul>
            <li> TF-Model </li>
                <code> return tf.train.GradientDescentOptimizer(0.8).minimize(self.delta_op, global_step = self.global_step)</code>
        </ul>
</ul>

```python
def get_name(self)
```
<ul type="circle">
    <li>Gibt den Namen dieses Neuronalen Netzes als String zurück, z.B. __TD-Gammon__, __TF-Gammon__, __Net5000__, etc.</li>
</ul>

In [49]:
from abc import ABC, abstractmethod
from Player import RandomPlayer, TDGammonPlayer
from FasterBackgammon import Game
import tensorflow as tf
import numpy as np
import random
import os

class NeuralNetModel(ABC):
    
    def __init__(self, sess, restore=False):
        #Speicherort ermitteln
        self.checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'checkpoints/' + self.get_name() +'/')
        
        #session
        self.sess = sess
        self.global_step = tf.Variable(0, trainable=False, name='global_step')

        #Für leichteres abändern der Netzwerkarchitektur
        layer_size_input = 198
        layer_size_hidden = 40
        layer_size_output = 1

        # placeholders for input and target output
        self.x = tf.placeholder('float', [1, layer_size_input], name='x')
        self.V_next = tf.placeholder('float', [1, layer_size_output], name='V_next')

        #Zwei fully-connected, dense Layer: Ein Input-Layer (198 Units) und dann ein Hidden-Layer (40 Units)
        #Beide werden mit der sigmoid Funktion aktiviert
        prev_y = tf.layers.dense(self.x, layer_size_hidden, activation= tf.sigmoid)
        self.V = tf.layers.dense(prev_y, layer_size_output, activation= tf.sigmoid)
        
        # delta = V_next - V
        self.delta_op = tf.reduce_sum(self.V_next - self.V, name='delta')

        #Erstellt eine Trainings- Operation und speichert diese ab
        self.train_op = self.create_training_op()

        #Der Saver behält immer nur den neusten Checkpoint
        self.saver = tf.train.Saver(max_to_keep=1)

        #Variablen initialisieren 
        self.sess.run(tf.global_variables_initializer())

        #Modell wiederherstellen falls gewünscht
        if restore:
            self.restore()
            
    @abstractmethod
    def create_training_op(self):
        pass
    
    @abstractmethod
    def get_name(self):
        pass
    
    """
        Bekannte Methoden aus der TD-GammonModel Klasse (004)
    """
            
        #Lädt den neusten (und auch einzigen) checkpoint 
    def restore(self):
        latest_checkpoint_path = tf.train.latest_checkpoint(self.checkpoint_path)
        if latest_checkpoint_path:
            print("Restoring checkpoint: {0}".format(latest_checkpoint_path))
            self.saver.restore(self.sess, latest_checkpoint_path)

    #Erzeugt einen Outpt für die gegebenen features x
    def get_output(self, x):
        return self.sess.run(self.V, feed_dict={ self.x: x })

    #Testet das Modell gegen den angegebenen enemyAgent
    def test(self, enemyPlayer=RandomPlayer('white'), games=100, debug=False):
        players = [ModelPlayer('black', self), enemyPlayer]
        
        winners = {'black':0, 'white':0}
        for i in range(games):
            game = Game()

            winner = game.play(players, debug=debug)
            winners[winner] += 1

            winners_total = sum(winners.values())
            print("[Game %d] %s (%s) vs %s (%s) %d:%d of %d games (%.2f%%)" % (i, \
                players[0].get_name(), players[0].player, \
                players[1].get_name(), players[1].player, \
                winners['black'], winners['white'], winners_total, \
                (winners['black'] / winners_total) * 100.0))
            
    def train(self, games, validation_interval, test_games=100):
        #Selbsttraining, Modell vs Modell
        players = [ModelPlayer('black', self), ModelPlayer('white', self)]

        for i in range(games):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if i != 0 and i % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(games = test_games)

            #Spiel initialisieren
            game = Game()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run([self.train_op, self.delta_op], feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step = self.sess.run([
                self.train_op,
                self.global_step,
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (i, games, players[winner].player, game_step))
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step) 
        self.test(games = test_games)

### TF-Gammon

Nachdem man nun einiges an die Super-Klasse ausgelagert hat, lässt sich dieses Modell sehr kurz und übersichtlich schreiben:

In [50]:
class TFGammonModel(NeuralNetModel):

    def create_training_op(self):
        return tf.train.GradientDescentOptimizer(0.8).minimize(self.delta_op, global_step = self.global_step)
    
    def get_name(self):
        return "TF-Gammon"

In [51]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel(sess, restore=False)
    model.test(games = 200)

[Game 0] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 1:1 of 2 games (50.00%)
[Game 2] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 2:1 of 3 games (66.67%)
[Game 3] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 2:2 of 4 games (50.00%)
[Game 4] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 2:3 of 5 games (40.00%)
[Game 5] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 3:3 of 6 games (50.00%)
[Game 6] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 3:4 of 7 games (42.86%)
[Game 7] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 3:5 of 8 games (37.50%)
[Game 8] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 4:5 of 9 games (44.44%)
[Game 9] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 5:5 of 10 games (50.00%)
[Game 10] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 6:5 of 11 games (54.55%)
[Game 11] ModelPlay

[Game 89] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 37:53 of 90 games (41.11%)
[Game 90] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 37:54 of 91 games (40.66%)
[Game 91] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 38:54 of 92 games (41.30%)
[Game 92] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 39:54 of 93 games (41.94%)
[Game 93] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 39:55 of 94 games (41.49%)
[Game 94] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 40:55 of 95 games (42.11%)
[Game 95] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 40:56 of 96 games (41.67%)
[Game 96] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 40:57 of 97 games (41.24%)
[Game 97] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 41:57 of 98 games (41.84%)
[Game 98] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 42:57 of 99 games (42.42%)
[Game 99] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 43:5

[Game 176] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 69:108 of 177 games (38.98%)
[Game 177] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 70:108 of 178 games (39.33%)
[Game 178] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 70:109 of 179 games (39.11%)
[Game 179] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 70:110 of 180 games (38.89%)
[Game 180] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 70:111 of 181 games (38.67%)
[Game 181] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 70:112 of 182 games (38.46%)
[Game 182] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 71:112 of 183 games (38.80%)
[Game 183] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 71:113 of 184 games (38.59%)
[Game 184] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 71:114 of 185 games (38.38%)
[Game 185] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 71:115 of 186 games (38.17%)
[Game 186] ModelPlayer [TF-Gammon] (blac

Zufällige Gewichte sind zufällig

In [52]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel(sess, restore=False)
    model.train(5000,1000)

Game 0/5000 (Winner: white) in 65 turns
Game 1/5000 (Winner: black) in 106 turns
Game 2/5000 (Winner: white) in 42 turns
Game 3/5000 (Winner: white) in 53 turns
Game 4/5000 (Winner: black) in 57 turns
Game 5/5000 (Winner: black) in 81 turns
Game 6/5000 (Winner: white) in 84 turns
Game 7/5000 (Winner: white) in 99 turns
Game 8/5000 (Winner: black) in 56 turns
Game 9/5000 (Winner: black) in 76 turns
Game 10/5000 (Winner: white) in 55 turns
Game 11/5000 (Winner: white) in 53 turns
Game 12/5000 (Winner: white) in 107 turns
Game 13/5000 (Winner: black) in 63 turns
Game 14/5000 (Winner: black) in 68 turns
Game 15/5000 (Winner: white) in 82 turns
Game 16/5000 (Winner: white) in 101 turns
Game 17/5000 (Winner: black) in 106 turns
Game 18/5000 (Winner: black) in 57 turns
Game 19/5000 (Winner: black) in 54 turns
Game 20/5000 (Winner: black) in 76 turns
Game 21/5000 (Winner: white) in 62 turns
Game 22/5000 (Winner: black) in 84 turns
Game 23/5000 (Winner: white) in 61 turns
Game 24/5000 (Winner: 

Game 198/5000 (Winner: black) in 49 turns
Game 199/5000 (Winner: black) in 96 turns
Game 200/5000 (Winner: black) in 81 turns
Game 201/5000 (Winner: black) in 60 turns
Game 202/5000 (Winner: black) in 79 turns
Game 203/5000 (Winner: black) in 44 turns
Game 204/5000 (Winner: white) in 64 turns
Game 205/5000 (Winner: black) in 97 turns
Game 206/5000 (Winner: black) in 46 turns
Game 207/5000 (Winner: black) in 78 turns
Game 208/5000 (Winner: white) in 57 turns
Game 209/5000 (Winner: white) in 58 turns
Game 210/5000 (Winner: black) in 49 turns
Game 211/5000 (Winner: black) in 53 turns
Game 212/5000 (Winner: white) in 83 turns
Game 213/5000 (Winner: white) in 55 turns
Game 214/5000 (Winner: black) in 67 turns
Game 215/5000 (Winner: white) in 75 turns
Game 216/5000 (Winner: black) in 52 turns
Game 217/5000 (Winner: black) in 67 turns
Game 218/5000 (Winner: white) in 71 turns
Game 219/5000 (Winner: white) in 48 turns
Game 220/5000 (Winner: black) in 47 turns
Game 221/5000 (Winner: white) in 5

Game 393/5000 (Winner: black) in 67 turns
Game 394/5000 (Winner: black) in 73 turns
Game 395/5000 (Winner: black) in 44 turns
Game 396/5000 (Winner: white) in 61 turns
Game 397/5000 (Winner: black) in 60 turns
Game 398/5000 (Winner: black) in 73 turns
Game 399/5000 (Winner: white) in 61 turns
Game 400/5000 (Winner: white) in 53 turns
Game 401/5000 (Winner: black) in 95 turns
Game 402/5000 (Winner: black) in 52 turns
Game 403/5000 (Winner: white) in 105 turns
Game 404/5000 (Winner: black) in 49 turns
Game 405/5000 (Winner: white) in 94 turns
Game 406/5000 (Winner: black) in 55 turns
Game 407/5000 (Winner: black) in 45 turns
Game 408/5000 (Winner: black) in 45 turns
Game 409/5000 (Winner: white) in 71 turns
Game 410/5000 (Winner: black) in 58 turns
Game 411/5000 (Winner: black) in 71 turns
Game 412/5000 (Winner: black) in 64 turns
Game 413/5000 (Winner: white) in 55 turns
Game 414/5000 (Winner: white) in 47 turns
Game 415/5000 (Winner: black) in 99 turns
Game 416/5000 (Winner: white) in 

Game 588/5000 (Winner: white) in 94 turns
Game 589/5000 (Winner: black) in 47 turns
Game 590/5000 (Winner: white) in 42 turns
Game 591/5000 (Winner: white) in 82 turns
Game 592/5000 (Winner: white) in 56 turns
Game 593/5000 (Winner: white) in 44 turns
Game 594/5000 (Winner: white) in 50 turns
Game 595/5000 (Winner: white) in 86 turns
Game 596/5000 (Winner: white) in 44 turns
Game 597/5000 (Winner: white) in 65 turns
Game 598/5000 (Winner: black) in 68 turns
Game 599/5000 (Winner: black) in 74 turns
Game 600/5000 (Winner: white) in 46 turns
Game 601/5000 (Winner: white) in 68 turns
Game 602/5000 (Winner: white) in 75 turns
Game 603/5000 (Winner: white) in 53 turns
Game 604/5000 (Winner: black) in 70 turns
Game 605/5000 (Winner: white) in 64 turns
Game 606/5000 (Winner: black) in 58 turns
Game 607/5000 (Winner: black) in 43 turns
Game 608/5000 (Winner: black) in 72 turns
Game 609/5000 (Winner: white) in 42 turns
Game 610/5000 (Winner: white) in 74 turns
Game 611/5000 (Winner: black) in 5

Game 783/5000 (Winner: black) in 55 turns
Game 784/5000 (Winner: black) in 50 turns
Game 785/5000 (Winner: black) in 82 turns
Game 786/5000 (Winner: white) in 71 turns
Game 787/5000 (Winner: black) in 53 turns
Game 788/5000 (Winner: black) in 61 turns
Game 789/5000 (Winner: white) in 45 turns
Game 790/5000 (Winner: black) in 68 turns
Game 791/5000 (Winner: white) in 58 turns
Game 792/5000 (Winner: white) in 51 turns
Game 793/5000 (Winner: white) in 54 turns
Game 794/5000 (Winner: black) in 46 turns
Game 795/5000 (Winner: black) in 74 turns
Game 796/5000 (Winner: black) in 112 turns
Game 797/5000 (Winner: black) in 52 turns
Game 798/5000 (Winner: white) in 50 turns
Game 799/5000 (Winner: black) in 70 turns
Game 800/5000 (Winner: white) in 81 turns
Game 801/5000 (Winner: black) in 55 turns
Game 802/5000 (Winner: black) in 44 turns
Game 803/5000 (Winner: black) in 56 turns
Game 804/5000 (Winner: black) in 59 turns
Game 805/5000 (Winner: white) in 71 turns
Game 806/5000 (Winner: black) in 

...

Game 4323/5000 (Winner: white) in 65 turns
Game 4324/5000 (Winner: black) in 72 turns
Game 4325/5000 (Winner: white) in 88 turns
Game 4326/5000 (Winner: black) in 55 turns
Game 4327/5000 (Winner: black) in 77 turns
Game 4328/5000 (Winner: black) in 57 turns
Game 4329/5000 (Winner: white) in 66 turns
Game 4330/5000 (Winner: white) in 53 turns
Game 4331/5000 (Winner: black) in 71 turns
Game 4332/5000 (Winner: black) in 78 turns
Game 4333/5000 (Winner: white) in 54 turns
Game 4334/5000 (Winner: black) in 53 turns
Game 4335/5000 (Winner: black) in 78 turns
Game 4336/5000 (Winner: white) in 74 turns
Game 4337/5000 (Winner: black) in 69 turns
Game 4338/5000 (Winner: white) in 60 turns
Game 4339/5000 (Winner: black) in 91 turns
Game 4340/5000 (Winner: white) in 60 turns
Game 4341/5000 (Winner: white) in 69 turns
Game 4342/5000 (Winner: white) in 73 turns
Game 4343/5000 (Winner: black) in 96 turns
Game 4344/5000 (Winner: black) in 63 turns
Game 4345/5000 (Winner: black) in 59 turns
Game 4346/5

Game 4514/5000 (Winner: white) in 48 turns
Game 4515/5000 (Winner: white) in 57 turns
Game 4516/5000 (Winner: black) in 60 turns
Game 4517/5000 (Winner: white) in 96 turns
Game 4518/5000 (Winner: white) in 115 turns
Game 4519/5000 (Winner: white) in 57 turns
Game 4520/5000 (Winner: white) in 71 turns
Game 4521/5000 (Winner: white) in 66 turns
Game 4522/5000 (Winner: black) in 83 turns
Game 4523/5000 (Winner: white) in 96 turns
Game 4524/5000 (Winner: black) in 86 turns
Game 4525/5000 (Winner: black) in 156 turns
Game 4526/5000 (Winner: black) in 58 turns
Game 4527/5000 (Winner: white) in 97 turns
Game 4528/5000 (Winner: black) in 63 turns
Game 4529/5000 (Winner: black) in 47 turns
Game 4530/5000 (Winner: white) in 53 turns
Game 4531/5000 (Winner: white) in 51 turns
Game 4532/5000 (Winner: black) in 68 turns
Game 4533/5000 (Winner: black) in 51 turns
Game 4534/5000 (Winner: black) in 63 turns
Game 4535/5000 (Winner: black) in 93 turns
Game 4536/5000 (Winner: black) in 57 turns
Game 4537

Game 4705/5000 (Winner: white) in 82 turns
Game 4706/5000 (Winner: white) in 41 turns
Game 4707/5000 (Winner: black) in 68 turns
Game 4708/5000 (Winner: black) in 75 turns
Game 4709/5000 (Winner: white) in 46 turns
Game 4710/5000 (Winner: white) in 133 turns
Game 4711/5000 (Winner: white) in 61 turns
Game 4712/5000 (Winner: black) in 99 turns
Game 4713/5000 (Winner: black) in 119 turns
Game 4714/5000 (Winner: black) in 71 turns
Game 4715/5000 (Winner: white) in 96 turns
Game 4716/5000 (Winner: white) in 65 turns
Game 4717/5000 (Winner: black) in 99 turns
Game 4718/5000 (Winner: white) in 54 turns
Game 4719/5000 (Winner: white) in 63 turns
Game 4720/5000 (Winner: black) in 83 turns
Game 4721/5000 (Winner: white) in 55 turns
Game 4722/5000 (Winner: black) in 43 turns
Game 4723/5000 (Winner: white) in 41 turns
Game 4724/5000 (Winner: white) in 76 turns
Game 4725/5000 (Winner: black) in 64 turns
Game 4726/5000 (Winner: black) in 65 turns
Game 4727/5000 (Winner: white) in 69 turns
Game 4728

Game 4896/5000 (Winner: black) in 80 turns
Game 4897/5000 (Winner: black) in 55 turns
Game 4898/5000 (Winner: white) in 63 turns
Game 4899/5000 (Winner: black) in 53 turns
Game 4900/5000 (Winner: white) in 69 turns
Game 4901/5000 (Winner: white) in 73 turns
Game 4902/5000 (Winner: white) in 71 turns
Game 4903/5000 (Winner: white) in 58 turns
Game 4904/5000 (Winner: black) in 49 turns
Game 4905/5000 (Winner: black) in 64 turns
Game 4906/5000 (Winner: white) in 53 turns
Game 4907/5000 (Winner: black) in 45 turns
Game 4908/5000 (Winner: black) in 47 turns
Game 4909/5000 (Winner: white) in 43 turns
Game 4910/5000 (Winner: white) in 52 turns
Game 4911/5000 (Winner: black) in 68 turns
Game 4912/5000 (Winner: black) in 64 turns
Game 4913/5000 (Winner: white) in 52 turns
Game 4914/5000 (Winner: black) in 66 turns
Game 4915/5000 (Winner: black) in 103 turns
Game 4916/5000 (Winner: white) in 57 turns
Game 4917/5000 (Winner: black) in 74 turns
Game 4918/5000 (Winner: black) in 115 turns
Game 4919

[Game 43] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 34:10 of 44 games (77.27%)
[Game 44] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 35:10 of 45 games (77.78%)
[Game 45] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 35:11 of 46 games (76.09%)
[Game 46] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 36:11 of 47 games (76.60%)
[Game 47] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 37:11 of 48 games (77.08%)
[Game 48] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 38:11 of 49 games (77.55%)
[Game 49] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 39:11 of 50 games (78.00%)
[Game 50] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 40:11 of 51 games (78.43%)
[Game 51] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 40:12 of 52 games (76.92%)
[Game 52] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 41:12 of 53 games (77.36%)
[Game 53] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 42:1

## Ergebnis

Durch die hohe Lernrate ist das Delta sehr schnell minimiert, jedoch wird es nach 1000 Trainingsspielen nicht mehr kleiner und auch nach 5000 Spielen bleibt es gleich und das neuronale netz verbessert sich nicht mehr

Daher ist allein das Minimieren von Delta nicht der Weg um einen starken Spieler zu erschaffen

__TF gegen Singleton__

In [53]:
import Player

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel(sess, restore=True)
    model.test(games = 200, enemyPlayer = Player.ValuePlayer(Game.PLAYERS[1], Player.singleton))

Restoring checkpoint: checkpoints/TF-Gammon/checkpoint.ckpt-341277
INFO:tensorflow:Restoring parameters from checkpoints/TF-Gammon/checkpoint.ckpt-341277
[Game 0] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 1:1 of 2 games (50.00%)
[Game 2] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:1 of 3 games (66.67%)
[Game 3] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:2 of 4 games (50.00%)
[Game 4] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:3 of 5 games (40.00%)
[Game 5] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:4 of 6 games (33.33%)
[Game 6] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:5 of 7 games (28.57%)
[Game 7] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 2:6 of 8 games (25.00%)
[Game 8] ModelPlayer [TF-Gammon] (black) vs Val

[Game 79] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 31:49 of 80 games (38.75%)
[Game 80] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 32:49 of 81 games (39.51%)
[Game 81] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 32:50 of 82 games (39.02%)
[Game 82] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 33:50 of 83 games (39.76%)
[Game 83] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 33:51 of 84 games (39.29%)
[Game 84] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 33:52 of 85 games (38.82%)
[Game 85] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 34:52 of 86 games (39.53%)
[Game 86] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 34:53 of 87 games (39.08%)
[Game 87] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 35:53 of 88 games (39.77%)
[Game 88] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [single

[Game 157] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 62:96 of 158 games (39.24%)
[Game 158] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 62:97 of 159 games (38.99%)
[Game 159] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 63:97 of 160 games (39.38%)
[Game 160] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 63:98 of 161 games (39.13%)
[Game 161] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 63:99 of 162 games (38.89%)
[Game 162] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 63:100 of 163 games (38.65%)
[Game 163] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 64:100 of 164 games (39.02%)
[Game 164] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 65:100 of 165 games (39.39%)
[Game 165] ModelPlayer [TF-Gammon] (black) vs ValuePlayer [singleton] (white) 66:100 of 166 games (39.76%)
[Game 166] ModelPlayer [TF-Gammon] (black)

All in. 160 Hidden units!

In [1]:
import tensorflow as tf
from NeuralNetModel import TFGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel(sess, hidden_size = 160, name = "TF160", restore=False)
    model.train(20000,1000)

Game 0/20000 (Winner: white) in 53 turns
Game 1/20000 (Winner: black) in 82 turns
Game 2/20000 (Winner: black) in 113 turns
Game 3/20000 (Winner: white) in 109 turns
Game 4/20000 (Winner: black) in 370 turns
Game 5/20000 (Winner: black) in 356 turns
Game 6/20000 (Winner: white) in 313 turns
Game 7/20000 (Winner: white) in 120 turns
Game 8/20000 (Winner: black) in 237 turns
Game 9/20000 (Winner: white) in 173 turns
Game 10/20000 (Winner: white) in 103 turns
Game 11/20000 (Winner: black) in 107 turns
Game 12/20000 (Winner: white) in 101 turns
Game 13/20000 (Winner: black) in 330 turns
Game 14/20000 (Winner: black) in 148 turns
Game 15/20000 (Winner: black) in 114 turns
Game 16/20000 (Winner: white) in 158 turns
Game 17/20000 (Winner: black) in 290 turns
Game 18/20000 (Winner: black) in 304 turns
Game 19/20000 (Winner: black) in 127 turns
Game 20/20000 (Winner: black) in 259 turns
Game 21/20000 (Winner: white) in 189 turns
Game 22/20000 (Winner: black) in 285 turns
Game 23/20000 (Winner: 

Game 190/20000 (Winner: white) in 88 turns
Game 191/20000 (Winner: black) in 70 turns
Game 192/20000 (Winner: white) in 137 turns
Game 193/20000 (Winner: black) in 140 turns
Game 194/20000 (Winner: white) in 98 turns
Game 195/20000 (Winner: black) in 101 turns
Game 196/20000 (Winner: white) in 241 turns
Game 197/20000 (Winner: white) in 168 turns
Game 198/20000 (Winner: white) in 151 turns
Game 199/20000 (Winner: white) in 141 turns
Game 200/20000 (Winner: white) in 65 turns
Game 201/20000 (Winner: black) in 208 turns
Game 202/20000 (Winner: white) in 64 turns
Game 203/20000 (Winner: black) in 127 turns
Game 204/20000 (Winner: black) in 97 turns
Game 205/20000 (Winner: white) in 88 turns
Game 206/20000 (Winner: black) in 106 turns
Game 207/20000 (Winner: black) in 152 turns
Game 208/20000 (Winner: white) in 150 turns
Game 209/20000 (Winner: white) in 120 turns
Game 210/20000 (Winner: white) in 91 turns
Game 211/20000 (Winner: black) in 103 turns
Game 212/20000 (Winner: white) in 217 tu

Game 378/20000 (Winner: black) in 82 turns
Game 379/20000 (Winner: white) in 95 turns
Game 380/20000 (Winner: black) in 152 turns
Game 381/20000 (Winner: white) in 64 turns
Game 382/20000 (Winner: white) in 108 turns
Game 383/20000 (Winner: white) in 74 turns
Game 384/20000 (Winner: white) in 68 turns
Game 385/20000 (Winner: white) in 84 turns
Game 386/20000 (Winner: white) in 121 turns
Game 387/20000 (Winner: white) in 122 turns
Game 388/20000 (Winner: white) in 95 turns
Game 389/20000 (Winner: white) in 58 turns
Game 390/20000 (Winner: black) in 70 turns
Game 391/20000 (Winner: black) in 51 turns
Game 392/20000 (Winner: black) in 96 turns
Game 393/20000 (Winner: black) in 154 turns
Game 394/20000 (Winner: black) in 65 turns
Game 395/20000 (Winner: black) in 94 turns
Game 396/20000 (Winner: black) in 123 turns
Game 397/20000 (Winner: white) in 57 turns
Game 398/20000 (Winner: black) in 151 turns
Game 399/20000 (Winner: black) in 98 turns
Game 400/20000 (Winner: black) in 92 turns
Game

Game 567/20000 (Winner: black) in 63 turns
Game 568/20000 (Winner: black) in 96 turns
Game 569/20000 (Winner: white) in 144 turns
Game 570/20000 (Winner: black) in 66 turns
Game 571/20000 (Winner: black) in 66 turns
Game 572/20000 (Winner: black) in 77 turns
Game 573/20000 (Winner: black) in 178 turns
Game 574/20000 (Winner: white) in 102 turns
Game 575/20000 (Winner: white) in 142 turns
Game 576/20000 (Winner: black) in 67 turns
Game 577/20000 (Winner: black) in 163 turns
Game 578/20000 (Winner: white) in 141 turns
Game 579/20000 (Winner: black) in 98 turns
Game 580/20000 (Winner: black) in 111 turns
Game 581/20000 (Winner: white) in 105 turns
Game 582/20000 (Winner: white) in 75 turns
Game 583/20000 (Winner: white) in 67 turns
Game 584/20000 (Winner: white) in 73 turns
Game 585/20000 (Winner: white) in 180 turns
Game 586/20000 (Winner: black) in 122 turns
Game 587/20000 (Winner: white) in 107 turns
Game 588/20000 (Winner: black) in 58 turns
Game 589/20000 (Winner: black) in 122 turns

Game 756/20000 (Winner: white) in 98 turns
Game 757/20000 (Winner: white) in 145 turns
Game 758/20000 (Winner: white) in 103 turns
Game 759/20000 (Winner: white) in 180 turns
Game 760/20000 (Winner: black) in 91 turns
Game 761/20000 (Winner: black) in 89 turns
Game 762/20000 (Winner: black) in 162 turns
Game 763/20000 (Winner: black) in 149 turns
Game 764/20000 (Winner: white) in 72 turns
Game 765/20000 (Winner: black) in 132 turns
Game 766/20000 (Winner: black) in 81 turns
Game 767/20000 (Winner: black) in 112 turns
Game 768/20000 (Winner: white) in 139 turns
Game 769/20000 (Winner: white) in 57 turns
Game 770/20000 (Winner: black) in 126 turns
Game 771/20000 (Winner: white) in 65 turns
Game 772/20000 (Winner: white) in 105 turns
Game 773/20000 (Winner: black) in 109 turns
Game 774/20000 (Winner: black) in 131 turns
Game 775/20000 (Winner: black) in 92 turns
Game 776/20000 (Winner: white) in 81 turns
Game 777/20000 (Winner: white) in 127 turns
Game 778/20000 (Winner: white) in 109 tur

...

Game 9651/20000 (Winner: black) in 67 turns
Game 9652/20000 (Winner: white) in 96 turns
Game 9653/20000 (Winner: black) in 55 turns
Game 9654/20000 (Winner: white) in 73 turns
Game 9655/20000 (Winner: black) in 90 turns
Game 9656/20000 (Winner: black) in 55 turns
Game 9657/20000 (Winner: black) in 103 turns
Game 9658/20000 (Winner: black) in 73 turns
Game 9659/20000 (Winner: white) in 50 turns
Game 9660/20000 (Winner: black) in 82 turns
Game 9661/20000 (Winner: white) in 60 turns
Game 9662/20000 (Winner: black) in 80 turns
Game 9663/20000 (Winner: white) in 58 turns
Game 9664/20000 (Winner: black) in 125 turns
Game 9665/20000 (Winner: white) in 61 turns
Game 9666/20000 (Winner: white) in 110 turns
Game 9667/20000 (Winner: black) in 60 turns
Game 9668/20000 (Winner: black) in 64 turns
Game 9669/20000 (Winner: black) in 61 turns
Game 9670/20000 (Winner: black) in 85 turns
Game 9671/20000 (Winner: white) in 55 turns
Game 9672/20000 (Winner: white) in 76 turns
Game 9673/20000 (Winner: whit

Game 9837/20000 (Winner: black) in 84 turns
Game 9838/20000 (Winner: black) in 51 turns
Game 9839/20000 (Winner: black) in 84 turns
Game 9840/20000 (Winner: white) in 204 turns
Game 9841/20000 (Winner: white) in 87 turns
Game 9842/20000 (Winner: black) in 51 turns
Game 9843/20000 (Winner: white) in 47 turns
Game 9844/20000 (Winner: white) in 45 turns
Game 9845/20000 (Winner: white) in 93 turns
Game 9846/20000 (Winner: black) in 59 turns
Game 9847/20000 (Winner: white) in 56 turns
Game 9848/20000 (Winner: white) in 113 turns
Game 9849/20000 (Winner: black) in 94 turns
Game 9850/20000 (Winner: black) in 50 turns
Game 9851/20000 (Winner: black) in 55 turns
Game 9852/20000 (Winner: white) in 63 turns
Game 9853/20000 (Winner: black) in 78 turns
Game 9854/20000 (Winner: white) in 81 turns
Game 9855/20000 (Winner: white) in 114 turns
Game 9856/20000 (Winner: white) in 63 turns
Game 9857/20000 (Winner: black) in 59 turns
Game 9858/20000 (Winner: black) in 51 turns
Game 9859/20000 (Winner: blac

[Game 12] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:4 of 13 games (69.23%)
[Game 13] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:5 of 14 games (64.29%)
[Game 14] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:6 of 15 games (60.00%)
[Game 15] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:7 of 16 games (56.25%)
[Game 16] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:8 of 17 games (52.94%)
[Game 17] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 9:9 of 18 games (50.00%)
[Game 18] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 10:9 of 19 games (52.63%)
[Game 19] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 11:9 of 20 games (55.00%)
[Game 20] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 12:9 of 21 games (57.14%)
[Game 21] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 13:9 of 22 games (59.09%)
[Game 22] ModelPlayer [TF-Gammon] (black) vs RandomPlayer (white) 14:9 of 23 games (60

Game 10001/20000 (Winner: white) in 76 turns
Game 10002/20000 (Winner: black) in 86 turns
Game 10003/20000 (Winner: white) in 65 turns
Game 10004/20000 (Winner: white) in 55 turns
Game 10005/20000 (Winner: black) in 120 turns
Game 10006/20000 (Winner: black) in 176 turns
Game 10007/20000 (Winner: white) in 72 turns
Game 10008/20000 (Winner: white) in 98 turns
Game 10009/20000 (Winner: white) in 54 turns
Game 10010/20000 (Winner: black) in 57 turns
Game 10011/20000 (Winner: white) in 53 turns
Game 10012/20000 (Winner: white) in 61 turns
Game 10013/20000 (Winner: black) in 76 turns
Game 10014/20000 (Winner: white) in 90 turns
Game 10015/20000 (Winner: black) in 87 turns
Game 10016/20000 (Winner: black) in 81 turns
Game 10017/20000 (Winner: black) in 74 turns
Game 10018/20000 (Winner: black) in 130 turns
Game 10019/20000 (Winner: white) in 42 turns
Game 10020/20000 (Winner: black) in 60 turns
Game 10021/20000 (Winner: black) in 89 turns
Game 10022/20000 (Winner: white) in 149 turns
Game 1

KeyboardInterrupt: 

Bereits nach 1000 Spielen wird das maximale Potential erreicht und der Random-Player immer noch nicht nach 10000 Spielen besiegt. 

In [12]:
import Player
from NeuralNetModel import TFGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel(sess, hidden_size = 160, name = "TF160", restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TFGammonModel(sess2, restore=True)
        model.test(games = 200, enemyPlayer = Player.ModelPlayer('white', model2))

Restoring checkpoint: checkpoints/TF160/checkpoint.ckpt-886493
INFO:tensorflow:Restoring parameters from checkpoints/TF160/checkpoint.ckpt-886493
Restoring checkpoint: checkpoints/TF-Gammon/checkpoint.ckpt-341277
INFO:tensorflow:Restoring parameters from checkpoints/TF-Gammon/checkpoint.ckpt-341277
[Game 0] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 1:1 of 2 games (50.00%)
[Game 2] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 2:1 of 3 games (66.67%)
[Game 3] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 3:1 of 4 games (75.00%)
[Game 4] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 4:1 of 5 games (80.00%)
[Game 5] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 4:2 of 6 games (66.67%)
[Game 6] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 4:3 of 7 games (57.14%)
[

[Game 77] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 42:36 of 78 games (53.85%)
[Game 78] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 43:36 of 79 games (54.43%)
[Game 79] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 44:36 of 80 games (55.00%)
[Game 80] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:36 of 81 games (55.56%)
[Game 81] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:37 of 82 games (54.88%)
[Game 82] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:38 of 83 games (54.22%)
[Game 83] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:39 of 84 games (53.57%)
[Game 84] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:40 of 85 games (52.94%)
[Game 85] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 45:41 of 86 games (52.33%)
[Game 86] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gam

[Game 155] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 71:85 of 156 games (45.51%)
[Game 156] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 72:85 of 157 games (45.86%)
[Game 157] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 73:85 of 158 games (46.20%)
[Game 158] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 73:86 of 159 games (45.91%)
[Game 159] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 74:86 of 160 games (46.25%)
[Game 160] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 74:87 of 161 games (45.96%)
[Game 161] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 74:88 of 162 games (45.68%)
[Game 162] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 74:89 of 163 games (45.40%)
[Game 163] ModelPlayer [TF-Gammon] (black) vs ModelPlayer [TF-Gammon] (white) 74:90 of 164 games (45.12%)
[Game 164] ModelPlayer [TF-Gammon] (black) vs 

Mit 160 Hidden units ist das Modell dem 40 units Modell gleichwertig

### TF-Gammon2

Alternative Idee: Mean squared error minimieren

In [5]:
from NeuralNetModel import NeuralNetModel

class TFGammonModel2(NeuralNetModel):

    def create_training_op(self):
        squared_error = tf.reduce_mean(tf.squared_difference(self.V_next, self.V))
        return tf.train.GradientDescentOptimizer(0.8).minimize(squared_error, global_step = self.global_step)
    
    def get_name(self):
        return "TF-Gammon2"

In [6]:
import tensorflow as tf

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel2(sess, restore=False)
    model.train(5000,1000)

Game 0/5000 (Winner: white) in 191 turns
Game 1/5000 (Winner: black) in 132 turns
Game 2/5000 (Winner: black) in 65 turns
Game 3/5000 (Winner: black) in 52 turns
Game 4/5000 (Winner: black) in 68 turns
Game 5/5000 (Winner: white) in 64 turns
Game 6/5000 (Winner: white) in 87 turns
Game 7/5000 (Winner: black) in 117 turns
Game 8/5000 (Winner: white) in 68 turns
Game 9/5000 (Winner: white) in 60 turns
Game 10/5000 (Winner: black) in 75 turns
Game 11/5000 (Winner: white) in 59 turns
Game 12/5000 (Winner: black) in 88 turns
Game 13/5000 (Winner: white) in 40 turns
Game 14/5000 (Winner: white) in 43 turns
Game 15/5000 (Winner: white) in 81 turns
Game 16/5000 (Winner: white) in 55 turns
Game 17/5000 (Winner: black) in 60 turns
Game 18/5000 (Winner: black) in 63 turns
Game 19/5000 (Winner: black) in 55 turns
Game 20/5000 (Winner: black) in 44 turns
Game 21/5000 (Winner: white) in 57 turns
Game 22/5000 (Winner: black) in 47 turns
Game 23/5000 (Winner: black) in 54 turns
Game 24/5000 (Winner: w

Game 198/5000 (Winner: black) in 50 turns
Game 199/5000 (Winner: black) in 45 turns
Game 200/5000 (Winner: black) in 51 turns
Game 201/5000 (Winner: white) in 47 turns
Game 202/5000 (Winner: black) in 55 turns
Game 203/5000 (Winner: white) in 50 turns
Game 204/5000 (Winner: white) in 50 turns
Game 205/5000 (Winner: white) in 45 turns
Game 206/5000 (Winner: white) in 54 turns
Game 207/5000 (Winner: black) in 48 turns
Game 208/5000 (Winner: white) in 43 turns
Game 209/5000 (Winner: white) in 54 turns
Game 210/5000 (Winner: white) in 45 turns
Game 211/5000 (Winner: black) in 38 turns
Game 212/5000 (Winner: black) in 39 turns
Game 213/5000 (Winner: black) in 50 turns
Game 214/5000 (Winner: black) in 51 turns
Game 215/5000 (Winner: black) in 49 turns
Game 216/5000 (Winner: white) in 52 turns
Game 217/5000 (Winner: white) in 54 turns
Game 218/5000 (Winner: white) in 47 turns
Game 219/5000 (Winner: black) in 50 turns
Game 220/5000 (Winner: white) in 46 turns
Game 221/5000 (Winner: black) in 4

Game 394/5000 (Winner: black) in 65 turns
Game 395/5000 (Winner: black) in 54 turns
Game 396/5000 (Winner: white) in 46 turns
Game 397/5000 (Winner: white) in 51 turns
Game 398/5000 (Winner: white) in 47 turns
Game 399/5000 (Winner: white) in 42 turns
Game 400/5000 (Winner: white) in 43 turns
Game 401/5000 (Winner: white) in 43 turns
Game 402/5000 (Winner: white) in 53 turns
Game 403/5000 (Winner: white) in 44 turns
Game 404/5000 (Winner: white) in 52 turns
Game 405/5000 (Winner: white) in 55 turns
Game 406/5000 (Winner: white) in 54 turns
Game 407/5000 (Winner: black) in 45 turns
Game 408/5000 (Winner: white) in 60 turns
Game 409/5000 (Winner: black) in 48 turns
Game 410/5000 (Winner: black) in 48 turns
Game 411/5000 (Winner: white) in 43 turns
Game 412/5000 (Winner: black) in 57 turns
Game 413/5000 (Winner: black) in 44 turns
Game 414/5000 (Winner: black) in 50 turns
Game 415/5000 (Winner: black) in 41 turns
Game 416/5000 (Winner: white) in 50 turns
Game 417/5000 (Winner: white) in 4

Game 590/5000 (Winner: black) in 59 turns
Game 591/5000 (Winner: white) in 46 turns
Game 592/5000 (Winner: black) in 40 turns
Game 593/5000 (Winner: white) in 44 turns
Game 594/5000 (Winner: black) in 51 turns
Game 595/5000 (Winner: white) in 46 turns
Game 596/5000 (Winner: white) in 44 turns
Game 597/5000 (Winner: black) in 50 turns
Game 598/5000 (Winner: white) in 49 turns
Game 599/5000 (Winner: white) in 48 turns
Game 600/5000 (Winner: white) in 58 turns
Game 601/5000 (Winner: black) in 53 turns
Game 602/5000 (Winner: white) in 51 turns
Game 603/5000 (Winner: black) in 51 turns
Game 604/5000 (Winner: black) in 68 turns
Game 605/5000 (Winner: white) in 44 turns
Game 606/5000 (Winner: white) in 50 turns
Game 607/5000 (Winner: black) in 56 turns
Game 608/5000 (Winner: black) in 54 turns
Game 609/5000 (Winner: white) in 44 turns
Game 610/5000 (Winner: white) in 70 turns
Game 611/5000 (Winner: white) in 56 turns
Game 612/5000 (Winner: white) in 62 turns
Game 613/5000 (Winner: white) in 5

Game 786/5000 (Winner: white) in 46 turns
Game 787/5000 (Winner: black) in 60 turns
Game 788/5000 (Winner: black) in 51 turns
Game 789/5000 (Winner: white) in 61 turns
Game 790/5000 (Winner: black) in 60 turns
Game 791/5000 (Winner: black) in 50 turns
Game 792/5000 (Winner: black) in 35 turns
Game 793/5000 (Winner: white) in 57 turns
Game 794/5000 (Winner: black) in 50 turns
Game 795/5000 (Winner: black) in 49 turns
Game 796/5000 (Winner: black) in 51 turns
Game 797/5000 (Winner: black) in 57 turns
Game 798/5000 (Winner: black) in 61 turns
Game 799/5000 (Winner: black) in 46 turns
Game 800/5000 (Winner: white) in 52 turns
Game 801/5000 (Winner: white) in 55 turns
Game 802/5000 (Winner: white) in 86 turns
Game 803/5000 (Winner: black) in 63 turns
Game 804/5000 (Winner: black) in 51 turns
Game 805/5000 (Winner: black) in 43 turns
Game 806/5000 (Winner: black) in 57 turns
Game 807/5000 (Winner: white) in 39 turns
Game 808/5000 (Winner: black) in 68 turns
Game 809/5000 (Winner: white) in 7

...

Game 4305/5000 (Winner: white) in 105 turns
Game 4306/5000 (Winner: black) in 64 turns
Game 4307/5000 (Winner: black) in 77 turns
Game 4308/5000 (Winner: white) in 59 turns
Game 4309/5000 (Winner: black) in 115 turns
Game 4310/5000 (Winner: white) in 141 turns
Game 4311/5000 (Winner: black) in 90 turns
Game 4312/5000 (Winner: white) in 67 turns
Game 4313/5000 (Winner: white) in 72 turns
Game 4314/5000 (Winner: white) in 128 turns
Game 4315/5000 (Winner: black) in 63 turns
Game 4316/5000 (Winner: black) in 157 turns
Game 4317/5000 (Winner: black) in 83 turns
Game 4318/5000 (Winner: white) in 77 turns
Game 4319/5000 (Winner: black) in 68 turns
Game 4320/5000 (Winner: white) in 66 turns
Game 4321/5000 (Winner: white) in 156 turns
Game 4322/5000 (Winner: white) in 89 turns
Game 4323/5000 (Winner: white) in 57 turns
Game 4324/5000 (Winner: white) in 121 turns
Game 4325/5000 (Winner: white) in 131 turns
Game 4326/5000 (Winner: black) in 115 turns
Game 4327/5000 (Winner: white) in 123 turns
G

Game 4494/5000 (Winner: white) in 117 turns
Game 4495/5000 (Winner: black) in 86 turns
Game 4496/5000 (Winner: white) in 80 turns
Game 4497/5000 (Winner: white) in 101 turns
Game 4498/5000 (Winner: white) in 60 turns
Game 4499/5000 (Winner: black) in 89 turns
Game 4500/5000 (Winner: white) in 77 turns
Game 4501/5000 (Winner: black) in 85 turns
Game 4502/5000 (Winner: white) in 117 turns
Game 4503/5000 (Winner: white) in 91 turns
Game 4504/5000 (Winner: white) in 112 turns
Game 4505/5000 (Winner: white) in 118 turns
Game 4506/5000 (Winner: white) in 175 turns
Game 4507/5000 (Winner: black) in 108 turns
Game 4508/5000 (Winner: black) in 82 turns
Game 4509/5000 (Winner: white) in 89 turns
Game 4510/5000 (Winner: black) in 60 turns
Game 4511/5000 (Winner: white) in 75 turns
Game 4512/5000 (Winner: black) in 80 turns
Game 4513/5000 (Winner: white) in 110 turns
Game 4514/5000 (Winner: white) in 63 turns
Game 4515/5000 (Winner: black) in 125 turns
Game 4516/5000 (Winner: white) in 109 turns
G

Game 4683/5000 (Winner: black) in 65 turns
Game 4684/5000 (Winner: white) in 82 turns
Game 4685/5000 (Winner: black) in 91 turns
Game 4686/5000 (Winner: black) in 69 turns
Game 4687/5000 (Winner: white) in 68 turns
Game 4688/5000 (Winner: white) in 49 turns
Game 4689/5000 (Winner: white) in 65 turns
Game 4690/5000 (Winner: black) in 158 turns
Game 4691/5000 (Winner: black) in 53 turns
Game 4692/5000 (Winner: white) in 90 turns
Game 4693/5000 (Winner: white) in 57 turns
Game 4694/5000 (Winner: white) in 93 turns
Game 4695/5000 (Winner: white) in 79 turns
Game 4696/5000 (Winner: black) in 112 turns
Game 4697/5000 (Winner: white) in 82 turns
Game 4698/5000 (Winner: black) in 99 turns
Game 4699/5000 (Winner: white) in 137 turns
Game 4700/5000 (Winner: black) in 74 turns
Game 4701/5000 (Winner: white) in 169 turns
Game 4702/5000 (Winner: black) in 75 turns
Game 4703/5000 (Winner: black) in 101 turns
Game 4704/5000 (Winner: black) in 62 turns
Game 4705/5000 (Winner: white) in 107 turns
Game 

Game 4873/5000 (Winner: white) in 76 turns
Game 4874/5000 (Winner: white) in 63 turns
Game 4875/5000 (Winner: white) in 55 turns
Game 4876/5000 (Winner: white) in 112 turns
Game 4877/5000 (Winner: black) in 105 turns
Game 4878/5000 (Winner: white) in 66 turns
Game 4879/5000 (Winner: white) in 135 turns
Game 4880/5000 (Winner: white) in 82 turns
Game 4881/5000 (Winner: white) in 84 turns
Game 4882/5000 (Winner: black) in 140 turns
Game 4883/5000 (Winner: black) in 52 turns
Game 4884/5000 (Winner: white) in 76 turns
Game 4885/5000 (Winner: white) in 89 turns
Game 4886/5000 (Winner: black) in 97 turns
Game 4887/5000 (Winner: white) in 112 turns
Game 4888/5000 (Winner: white) in 97 turns
Game 4889/5000 (Winner: black) in 119 turns
Game 4890/5000 (Winner: black) in 107 turns
Game 4891/5000 (Winner: white) in 108 turns
Game 4892/5000 (Winner: black) in 56 turns
Game 4893/5000 (Winner: white) in 71 turns
Game 4894/5000 (Winner: black) in 120 turns
Game 4895/5000 (Winner: black) in 96 turns
Ga

[Game 29] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 30:0 of 30 games (100.00%)
[Game 30] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 31:0 of 31 games (100.00%)
[Game 31] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 32:0 of 32 games (100.00%)
[Game 32] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 33:0 of 33 games (100.00%)
[Game 33] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 34:0 of 34 games (100.00%)
[Game 34] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 35:0 of 35 games (100.00%)
[Game 35] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 36:0 of 36 games (100.00%)
[Game 36] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 37:0 of 37 games (100.00%)
[Game 37] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 38:0 of 38 games (100.00%)
[Game 38] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 39:0 of 39 games (100.00%)
[Game 39] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (

### TD-Gammon vs TF-Gammon

In [8]:
import Player
import PlayerTest
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TFGammonModel2(sess2, restore=True)
        players = [Player.ModelPlayer('black', model), Player.ModelPlayer('white', model2)]
        PlayerTest.test(players, games = 1000)

Restoring checkpoint: checkpoints/TD-Gammon/checkpoint.ckpt-1593683
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon/checkpoint.ckpt-1593683
Restoring checkpoint: checkpoints/TF-Gammon2/checkpoint.ckpt-404677
INFO:tensorflow:Restoring parameters from checkpoints/TF-Gammon2/checkpoint.ckpt-404677
Spiel 0 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 1 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 2 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 3 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 4 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 5 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 6 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 7 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 8 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 9 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 10 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 11 von 1000 geht an Model

Spiel 131 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 132 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 133 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 134 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 135 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 136 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 137 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 138 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 139 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 140 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 141 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 142 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 143 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 144 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 145 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 146 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 147 von 10

Spiel 265 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 266 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 267 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 268 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 269 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 270 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 271 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 272 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 273 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 274 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 275 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 276 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 277 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 278 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 279 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 280 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 281 von 1

Spiel 399 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 400 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 401 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 402 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 403 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 404 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 405 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 406 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 407 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 408 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 409 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 410 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 411 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 412 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 413 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 414 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 415 von 

Spiel 533 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 534 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 535 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 536 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 537 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 538 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 539 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 540 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 541 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 542 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 543 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 544 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 545 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 546 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 547 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 548 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 549 vo

Spiel 666 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 667 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 668 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 669 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 670 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 671 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 672 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 673 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 674 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 675 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 676 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 677 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 678 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 679 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 680 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 681 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 682 von 10

Spiel 800 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 801 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 802 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 803 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 804 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 805 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 806 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 807 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 808 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 809 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 810 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 811 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 812 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 813 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 814 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 815 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 816 von 

Spiel 934 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 935 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 936 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 937 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 938 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 939 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 940 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 941 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 942 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 943 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 944 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 945 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 946 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 947 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 948 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 949 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 950 von 10

### TD-Gammon80 vs. TF-Gammon

In [9]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, hidden_size = 80, name = "TD-Gammon80",restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TFGammonModel2(sess2, restore=True)
        players = [Player.ModelPlayer('black', model), Player.ModelPlayer('white', model2)]
        PlayerTest.test(players, games = 1000)

Restoring checkpoint: checkpoints/TD-Gammon80/checkpoint.ckpt-1527600
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon80/checkpoint.ckpt-1527600
Restoring checkpoint: checkpoints/TF-Gammon2/checkpoint.ckpt-404677
INFO:tensorflow:Restoring parameters from checkpoints/TF-Gammon2/checkpoint.ckpt-404677
Spiel 0 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 1 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 2 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 3 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 4 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 5 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 6 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 7 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 8 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 9 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 10 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 11 von 1000 geht an Mo

Spiel 130 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 131 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 132 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 133 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 134 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 135 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 136 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 137 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 138 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 139 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 140 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 141 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 142 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 143 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 144 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 145 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 146 von 1000 

Spiel 264 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 265 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 266 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 267 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 268 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 269 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 270 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 271 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 272 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 273 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 274 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 275 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 276 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 277 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 278 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 279 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 280 von 100

Spiel 398 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 399 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 400 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 401 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 402 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 403 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 404 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 405 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 406 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 407 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 408 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 409 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 410 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 411 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 412 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 413 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 414 von 1000

Spiel 532 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 533 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 534 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 535 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 536 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 537 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 538 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 539 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 540 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 541 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 542 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 543 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 544 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 545 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 546 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 547 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 548 von 1000

Spiel 666 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 667 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 668 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 669 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 670 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 671 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 672 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 673 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 674 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 675 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 676 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 677 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 678 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 679 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 680 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 681 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 682 von 100

Spiel 800 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 801 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 802 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 803 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 804 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 805 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 806 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 807 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 808 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 809 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 810 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 811 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 812 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 813 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 814 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 815 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 816 von 1

Spiel 934 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 935 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 936 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 937 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 938 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 939 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 940 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 941 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 942 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 943 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 944 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 945 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 946 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 947 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 948 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 949 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 950 von 1000 

Mithilfe des quadrierten Fehlers steigt die Leistung von TD-Gammon erheblich

In [10]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TFGammonModel2(sess, hidden_size = 80, name = "TF-Gammon80", restore=False)
    model.train(20000,1000)

Game 0/20000 (Winner: black) in 198 turns
Game 1/20000 (Winner: black) in 142 turns
Game 2/20000 (Winner: white) in 216 turns
Game 3/20000 (Winner: white) in 165 turns
Game 4/20000 (Winner: black) in 46 turns
Game 5/20000 (Winner: white) in 103 turns
Game 6/20000 (Winner: white) in 298 turns
Game 7/20000 (Winner: white) in 238 turns
Game 8/20000 (Winner: black) in 91 turns
Game 9/20000 (Winner: black) in 81 turns
Game 10/20000 (Winner: white) in 43 turns
Game 11/20000 (Winner: white) in 58 turns
Game 12/20000 (Winner: white) in 88 turns
Game 13/20000 (Winner: black) in 84 turns
Game 14/20000 (Winner: white) in 56 turns
Game 15/20000 (Winner: black) in 60 turns
Game 16/20000 (Winner: black) in 49 turns
Game 17/20000 (Winner: white) in 46 turns
Game 18/20000 (Winner: black) in 49 turns
Game 19/20000 (Winner: white) in 53 turns
Game 20/20000 (Winner: white) in 57 turns
Game 21/20000 (Winner: white) in 73 turns
Game 22/20000 (Winner: black) in 60 turns
Game 23/20000 (Winner: white) in 59 t

Game 193/20000 (Winner: black) in 51 turns
Game 194/20000 (Winner: white) in 40 turns
Game 195/20000 (Winner: black) in 54 turns
Game 196/20000 (Winner: white) in 50 turns
Game 197/20000 (Winner: white) in 50 turns
Game 198/20000 (Winner: black) in 54 turns
Game 199/20000 (Winner: white) in 54 turns
Game 200/20000 (Winner: white) in 64 turns
Game 201/20000 (Winner: white) in 42 turns
Game 202/20000 (Winner: black) in 52 turns
Game 203/20000 (Winner: black) in 62 turns
Game 204/20000 (Winner: black) in 62 turns
Game 205/20000 (Winner: black) in 47 turns
Game 206/20000 (Winner: white) in 49 turns
Game 207/20000 (Winner: white) in 51 turns
Game 208/20000 (Winner: black) in 50 turns
Game 209/20000 (Winner: black) in 43 turns
Game 210/20000 (Winner: white) in 48 turns
Game 211/20000 (Winner: white) in 54 turns
Game 212/20000 (Winner: white) in 39 turns
Game 213/20000 (Winner: white) in 50 turns
Game 214/20000 (Winner: black) in 55 turns
Game 215/20000 (Winner: white) in 50 turns
Game 216/20

Game 385/20000 (Winner: white) in 55 turns
Game 386/20000 (Winner: white) in 53 turns
Game 387/20000 (Winner: white) in 41 turns
Game 388/20000 (Winner: white) in 54 turns
Game 389/20000 (Winner: white) in 48 turns
Game 390/20000 (Winner: black) in 52 turns
Game 391/20000 (Winner: black) in 50 turns
Game 392/20000 (Winner: white) in 55 turns
Game 393/20000 (Winner: white) in 48 turns
Game 394/20000 (Winner: black) in 56 turns
Game 395/20000 (Winner: white) in 48 turns
Game 396/20000 (Winner: white) in 59 turns
Game 397/20000 (Winner: white) in 41 turns
Game 398/20000 (Winner: white) in 56 turns
Game 399/20000 (Winner: white) in 58 turns
Game 400/20000 (Winner: white) in 53 turns
Game 401/20000 (Winner: white) in 48 turns
Game 402/20000 (Winner: white) in 55 turns
Game 403/20000 (Winner: white) in 54 turns
Game 404/20000 (Winner: black) in 59 turns
Game 405/20000 (Winner: white) in 56 turns
Game 406/20000 (Winner: white) in 44 turns
Game 407/20000 (Winner: black) in 44 turns
Game 408/20

Game 577/20000 (Winner: white) in 50 turns
Game 578/20000 (Winner: white) in 45 turns
Game 579/20000 (Winner: black) in 52 turns
Game 580/20000 (Winner: black) in 42 turns
Game 581/20000 (Winner: white) in 60 turns
Game 582/20000 (Winner: black) in 40 turns
Game 583/20000 (Winner: white) in 46 turns
Game 584/20000 (Winner: white) in 48 turns
Game 585/20000 (Winner: white) in 50 turns
Game 586/20000 (Winner: white) in 62 turns
Game 587/20000 (Winner: black) in 50 turns
Game 588/20000 (Winner: black) in 52 turns
Game 589/20000 (Winner: black) in 48 turns
Game 590/20000 (Winner: white) in 51 turns
Game 591/20000 (Winner: white) in 45 turns
Game 592/20000 (Winner: black) in 63 turns
Game 593/20000 (Winner: black) in 49 turns
Game 594/20000 (Winner: black) in 52 turns
Game 595/20000 (Winner: white) in 52 turns
Game 596/20000 (Winner: white) in 50 turns
Game 597/20000 (Winner: white) in 50 turns
Game 598/20000 (Winner: black) in 52 turns
Game 599/20000 (Winner: black) in 46 turns
Game 600/20

Game 768/20000 (Winner: black) in 51 turns
Game 769/20000 (Winner: black) in 63 turns
Game 770/20000 (Winner: white) in 43 turns
Game 771/20000 (Winner: white) in 43 turns
Game 772/20000 (Winner: white) in 56 turns
Game 773/20000 (Winner: white) in 48 turns
Game 774/20000 (Winner: white) in 44 turns
Game 775/20000 (Winner: black) in 49 turns
Game 776/20000 (Winner: white) in 41 turns
Game 777/20000 (Winner: white) in 51 turns
Game 778/20000 (Winner: white) in 42 turns
Game 779/20000 (Winner: white) in 64 turns
Game 780/20000 (Winner: black) in 59 turns
Game 781/20000 (Winner: white) in 54 turns
Game 782/20000 (Winner: white) in 49 turns
Game 783/20000 (Winner: white) in 45 turns
Game 784/20000 (Winner: white) in 45 turns
Game 785/20000 (Winner: white) in 52 turns
Game 786/20000 (Winner: white) in 55 turns
Game 787/20000 (Winner: white) in 55 turns
Game 788/20000 (Winner: white) in 43 turns
Game 789/20000 (Winner: white) in 41 turns
Game 790/20000 (Winner: black) in 60 turns
Game 791/20

...

Game 19352/20000 (Winner: black) in 79 turns
Game 19353/20000 (Winner: white) in 105 turns
Game 19354/20000 (Winner: black) in 80 turns
Game 19355/20000 (Winner: black) in 155 turns
Game 19356/20000 (Winner: white) in 123 turns
Game 19357/20000 (Winner: white) in 80 turns
Game 19358/20000 (Winner: black) in 166 turns
Game 19359/20000 (Winner: black) in 79 turns
Game 19360/20000 (Winner: white) in 49 turns
Game 19361/20000 (Winner: black) in 127 turns
Game 19362/20000 (Winner: white) in 73 turns
Game 19363/20000 (Winner: white) in 86 turns
Game 19364/20000 (Winner: black) in 141 turns
Game 19365/20000 (Winner: white) in 141 turns
Game 19366/20000 (Winner: white) in 216 turns
Game 19367/20000 (Winner: white) in 102 turns
Game 19368/20000 (Winner: white) in 75 turns
Game 19369/20000 (Winner: black) in 84 turns
Game 19370/20000 (Winner: white) in 95 turns
Game 19371/20000 (Winner: black) in 64 turns
Game 19372/20000 (Winner: white) in 93 turns
Game 19373/20000 (Winner: white) in 83 turns
G

Game 19533/20000 (Winner: white) in 135 turns
Game 19534/20000 (Winner: black) in 124 turns
Game 19535/20000 (Winner: black) in 83 turns
Game 19536/20000 (Winner: white) in 86 turns
Game 19537/20000 (Winner: white) in 132 turns
Game 19538/20000 (Winner: white) in 120 turns
Game 19539/20000 (Winner: white) in 121 turns
Game 19540/20000 (Winner: white) in 81 turns
Game 19541/20000 (Winner: white) in 84 turns
Game 19542/20000 (Winner: black) in 108 turns
Game 19543/20000 (Winner: white) in 85 turns
Game 19544/20000 (Winner: black) in 81 turns
Game 19545/20000 (Winner: white) in 95 turns
Game 19546/20000 (Winner: black) in 61 turns
Game 19547/20000 (Winner: white) in 85 turns
Game 19548/20000 (Winner: white) in 108 turns
Game 19549/20000 (Winner: black) in 66 turns
Game 19550/20000 (Winner: white) in 104 turns
Game 19551/20000 (Winner: white) in 143 turns
Game 19552/20000 (Winner: white) in 75 turns
Game 19553/20000 (Winner: white) in 55 turns
Game 19554/20000 (Winner: black) in 156 turns


Game 19715/20000 (Winner: black) in 163 turns
Game 19716/20000 (Winner: white) in 177 turns
Game 19717/20000 (Winner: black) in 66 turns
Game 19718/20000 (Winner: black) in 70 turns
Game 19719/20000 (Winner: black) in 120 turns
Game 19720/20000 (Winner: black) in 166 turns
Game 19721/20000 (Winner: white) in 97 turns
Game 19722/20000 (Winner: white) in 126 turns
Game 19723/20000 (Winner: white) in 156 turns
Game 19724/20000 (Winner: black) in 77 turns
Game 19725/20000 (Winner: black) in 55 turns
Game 19726/20000 (Winner: white) in 98 turns
Game 19727/20000 (Winner: black) in 113 turns
Game 19728/20000 (Winner: white) in 133 turns
Game 19729/20000 (Winner: black) in 100 turns
Game 19730/20000 (Winner: white) in 108 turns
Game 19731/20000 (Winner: white) in 115 turns
Game 19732/20000 (Winner: white) in 140 turns
Game 19733/20000 (Winner: white) in 127 turns
Game 19734/20000 (Winner: white) in 120 turns
Game 19735/20000 (Winner: black) in 93 turns
Game 19736/20000 (Winner: white) in 147 t

Game 19896/20000 (Winner: black) in 91 turns
Game 19897/20000 (Winner: black) in 155 turns
Game 19898/20000 (Winner: white) in 88 turns
Game 19899/20000 (Winner: white) in 85 turns
Game 19900/20000 (Winner: white) in 170 turns
Game 19901/20000 (Winner: black) in 129 turns
Game 19902/20000 (Winner: black) in 76 turns
Game 19903/20000 (Winner: black) in 128 turns
Game 19904/20000 (Winner: black) in 104 turns
Game 19905/20000 (Winner: black) in 69 turns
Game 19906/20000 (Winner: white) in 84 turns
Game 19907/20000 (Winner: white) in 68 turns
Game 19908/20000 (Winner: white) in 43 turns
Game 19909/20000 (Winner: white) in 202 turns
Game 19910/20000 (Winner: white) in 151 turns
Game 19911/20000 (Winner: white) in 80 turns
Game 19912/20000 (Winner: black) in 144 turns
Game 19913/20000 (Winner: black) in 175 turns
Game 19914/20000 (Winner: black) in 75 turns
Game 19915/20000 (Winner: black) in 143 turns
Game 19916/20000 (Winner: white) in 166 turns
Game 19917/20000 (Winner: white) in 88 turns

[Game 38] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 39:0 of 39 games (100.00%)
[Game 39] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 40:0 of 40 games (100.00%)
[Game 40] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 41:0 of 41 games (100.00%)
[Game 41] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 42:0 of 42 games (100.00%)
[Game 42] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 43:0 of 43 games (100.00%)
[Game 43] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 44:0 of 44 games (100.00%)
[Game 44] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 45:0 of 45 games (100.00%)
[Game 45] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 46:0 of 46 games (100.00%)
[Game 46] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 47:0 of 47 games (100.00%)
[Game 47] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (white) 48:0 of 48 games (100.00%)
[Game 48] ModelPlayer [TF-Gammon2] (black) vs RandomPlayer (

20000 Spiele, nun gleichauf mit TD-Gammon80

### TD-Gammon80 vs. TF-Gammon80

In [12]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, hidden_size = 80, name = "TD-Gammon80",restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TFGammonModel2(sess2, hidden_size = 80, name = "TF-Gammon80", restore=True)
        players = [Player.ModelPlayer('black', model), Player.ModelPlayer('white', model2)]
        PlayerTest.test(players, games = 1000)

Restoring checkpoint: checkpoints/TD-Gammon80/checkpoint.ckpt-1527600
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon80/checkpoint.ckpt-1527600
Restoring checkpoint: checkpoints/TF-Gammon80/checkpoint.ckpt-1984549
INFO:tensorflow:Restoring parameters from checkpoints/TF-Gammon80/checkpoint.ckpt-1984549
Spiel 0 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 1 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 2 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 3 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 4 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 5 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 6 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 7 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 8 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 9 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 10 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 11 von 1000 geht an M

Spiel 130 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 131 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 132 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 133 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 134 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 135 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 136 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 137 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 138 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 139 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 140 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 141 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 142 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 143 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 144 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 145 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 146 von 1

Spiel 264 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 265 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 266 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 267 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 268 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 269 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 270 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 271 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 272 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 273 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 274 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 275 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 276 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 277 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 278 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 279 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 280 von 100

Spiel 398 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 399 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 400 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 401 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 402 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 403 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 404 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 405 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 406 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 407 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 408 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 409 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 410 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 411 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 412 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 413 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 414 von 100

Spiel 532 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 533 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 534 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 535 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 536 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 537 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 538 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 539 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 540 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 541 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 542 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 543 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 544 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 545 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 546 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 547 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 548 von 

Spiel 666 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 667 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 668 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 669 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 670 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 671 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 672 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 673 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 674 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 675 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 676 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 677 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 678 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 679 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 680 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 681 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 682 vo

Spiel 800 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 801 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 802 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 803 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 804 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 805 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 806 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 807 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 808 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 809 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 810 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 811 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 812 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 813 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 814 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 815 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 816 von

Spiel 934 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 935 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 936 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 937 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 938 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 939 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 940 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 941 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 942 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 943 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 944 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 945 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 946 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 947 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 948 von 1000 geht an ModelPlayer [TF-Gammon2] ( white )
Spiel 949 von 1000 geht an ModelPlayer [TD-Gammon] ( black )
Spiel 950 von 1000

TF-Gammon80 kommt schon sehr nah an TD-Gammon80 dran